In [1]:
# Importing necessary libraries

import praw
import pandas as pd

In [2]:
# Reddit API credentials

reddit = praw.Reddit(client_id="*****",               # your client id
                     client_secret= "*****",          # your client secret
                     user_agent= "*****")             # your user agent

In [3]:
# Subreddit to scrape

subreddit = reddit.subreddit('depression')

In [4]:
# Define lists to store data

data = []

In [5]:
# Scraping posts & Comments

for post in subreddit.hot(limit= None): 
    data.append({
        'Type': 'Post',
        'Post_id': post.id,
        'Title': post.title,
        'Author': post.author.name if post.author else 'Unknown',
        'Timestamp': pd.to_datetime(post.created_utc, unit='s'), # Convert timestamp to datetime
        'Text': post.selftext,
        'Score': post.score,
        'Total_comments': post.num_comments
    })

    # Check if the post has comments
    if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit= None)
        for comment in post.comments.list():
            data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0  # Comments don't have total comments attribute
            })


In [6]:
# Create pandas DataFrame for posts and comments

reddit_depression_df = pd.DataFrame(data)

In [7]:
reddit_depression_df.head()

,Type,Post_id,Title,Author,Timestamp,Text,Score,Total_comments
0,Post,doqwow,Our most-broken and least-understood rules is ...,SQLwitch,2019-10-29 14:52:02,We understand that most people who reply immed...,2364,177
1,Comment,doqwow,Our most-broken and least-understood rules is ...,scorpiontank27,2019-10-29 15:53:38,Understood and I apologise if I forget in the ...,236,0
2,Comment,doqwow,Our most-broken and least-understood rules is ...,Unknown,2019-10-29 15:53:39,[removed],63,0
3,Comment,doqwow,Our most-broken and least-understood rules is ...,BloodyClash1133,2019-11-02 10:11:35,Biggest Problem on private talks may be that y...,49,0
4,Comment,doqwow,Our most-broken and least-understood rules is ...,Unknown,2019-10-29 15:43:23,I have to agree with this. I know that people ...,48,0


In [8]:
reddit_depression_df.shape

(3859, 8)

In [9]:
# Save DataFrame as JSON

reddit_depression_df.to_json('Reddit_depression_df.json', orient='records', lines=True)